# Coursework 1

This notebook is intended to be used as a starting point for your experiments. The instructions can be found in the MLP2024_25_CW1_Spec.pdf (see Learn,  Assignment Submission, Coursework 1). The methods provided here are just helper functions. If you want more complex graphs such as side by side comparisons of different experiments you should learn more about matplotlib and implement them. Before each experiment remember to re-initialize neural network weights and reset the data providers so you get a properly initialized experiment. For each experiment try to keep most hyperparameters the same except the one under investigation so you can understand what the effects of each are.

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

def train_model_and_plot_stats(
        model, error, learning_rule, train_data, valid_data, num_epochs, stats_interval, notebook=True):
    
    # As well as monitoring the error over training also monitor classification
    # accuracy i.e. proportion of most-probable predicted classes being equal to targets
    data_monitors={'acc': lambda y, t: (y.argmax(-1) == t.argmax(-1)).mean()}

    # Use the created objects to initialise a new Optimiser instance.
    optimiser = Optimiser(
        model, error, learning_rule, train_data, valid_data, data_monitors, notebook=notebook)

    # Run the optimiser for num_epochs epochs (full passes through the training set)
    # printing statistics every epoch.
    stats, keys, run_time = optimiser.train(num_epochs=num_epochs, stats_interval=stats_interval)

    # Plot the change in the validation and training set error over training.
    fig_1 = plt.figure(figsize=(8, 4))
    ax_1 = fig_1.add_subplot(111)
    for k in ['error(train)', 'error(valid)']:
        ax_1.plot(np.arange(1, stats.shape[0]) * stats_interval, 
                  stats[1:, keys[k]], label=k)
    ax_1.legend(loc=0)
    ax_1.set_xlabel('Epoch number')
    ax_1.set_ylabel('Error')

    # Plot the change in the validation and training set accuracy over training.
    fig_2 = plt.figure(figsize=(8, 4))
    ax_2 = fig_2.add_subplot(111)
    for k in ['acc(train)', 'acc(valid)']:
        ax_2.plot(np.arange(1, stats.shape[0]) * stats_interval, 
                  stats[1:, keys[k]], label=k)
    ax_2.legend(loc=0)
    ax_2.set_xlabel('Epoch number')
    ax_2.set_xlabel('Accuracy')
    
    return stats, keys, run_time, fig_1, ax_1, fig_2, ax_2

In [2]:
# The below code will set up the data providers, random number
# generator and logger objects needed for training runs. As
# loading the data from file take a little while you generally
# will probably not want to reload the data providers on
# every training run. If you wish to reset their state you
# should instead use the .reset() method of the data providers.
import numpy as np
import logging
import sys
sys.path.append('..')
from mlp.data_providers import MNISTDataProvider, EMNISTDataProvider

# Seed a random number generator
seed = 11102019 
rng = np.random.RandomState(seed)
batch_size = 100
# Set up a logger object to print info about the training run to stdout
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.handlers = [logging.StreamHandler()]

# Create data provider objects for the MNIST data set
train_data = EMNISTDataProvider('train', batch_size=batch_size, rng=rng)
valid_data = EMNISTDataProvider('valid', batch_size=batch_size, rng=rng)

KeysView(NpzFile '/home/natmaw/Documents/ML_Practical/mlpractical_natmaws/data/emnist-train.npz' with keys: inputs, targets)
KeysView(NpzFile '/home/natmaw/Documents/ML_Practical/mlpractical_natmaws/data/emnist-valid.npz' with keys: inputs, targets)


In [3]:
# The model set up code below is provided as a starting point.
# You will probably want to add further code cells for the
# different experiments you run.

%pip install tqdm

from mlp.layers import AffineLayer, SoftmaxLayer, SigmoidLayer, ReluLayer
from mlp.errors import CrossEntropySoftmaxError
from mlp.models import MultipleLayerModel
from mlp.initialisers import ConstantInit, GlorotUniformInit
from mlp.learning_rules import AdamLearningRule
from mlp.optimisers import Optimiser

# Setup hyperparameters
learning_rate = 0.001
num_epochs = 100
stats_interval = 1
input_dim, output_dim, hidden_dim = 784, 47, 128

weights_init = GlorotUniformInit(rng=rng)
biases_init = ConstantInit(0.)

# Create model with ONE hidden layer
model = MultipleLayerModel([
    AffineLayer(input_dim, hidden_dim, weights_init, biases_init), # hidden layer
    ReluLayer(),
    AffineLayer(hidden_dim, output_dim, weights_init, biases_init) # output layer
])

error = CrossEntropySoftmaxError()
# Use a Adam learning rule
learning_rule = AdamLearningRule(learning_rate=learning_rate)

# Remember to use notebook=False when you write a script to be run in a terminal
_ = train_model_and_plot_stats(
    model, error, learning_rule, train_data, valid_data, num_epochs, stats_interval, notebook=True)

Note: you may need to restart the kernel to use updated packages.


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [11]:
# Create model with TWO hidden layers
model = MultipleLayerModel([
    AffineLayer(input_dim, hidden_dim, weights_init, biases_init), # first hidden layer
    ReluLayer(),
    AffineLayer(hidden_dim, hidden_dim, weights_init, biases_init), # second hidden layer
    ReluLayer(),
    AffineLayer(hidden_dim, output_dim, weights_init, biases_init) # output layer
])

error = CrossEntropySoftmaxError()
# Use a Adam learning rule
learning_rule = AdamLearningRule(learning_rate=learning_rate)

# Remember to use notebook=False when you write a script to be run in a terminal
_ = train_model_and_plot_stats(
    model, error, learning_rule, train_data, valid_data, num_epochs, stats_interval, notebook=True)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [12]:
# Define a function to train the model with varying hidden units and track statistics
def evaluate_varying_hidden_units(hidden_units_list, input_dim, output_dim, learning_rate, num_epochs, stats_interval, train_data, valid_data):
    results = {}
    
    for hidden_units in hidden_units_list:
        print(f"\nTraining model with {hidden_units} hidden units.")
        
        # Initialize weights and biases
        weights_init = GlorotUniformInit(rng=rng)
        biases_init = ConstantInit(0.)
        
        # Define model with the current number of hidden units
        model = MultipleLayerModel([
            AffineLayer(input_dim, hidden_units, weights_init, biases_init),  # Hidden layer
            ReluLayer(),
            AffineLayer(hidden_units, output_dim, weights_init, biases_init)  # Output layer
        ])
        
        # Set up the loss function (cross-entropy) and learning rule (Adam)
        error = CrossEntropySoftmaxError()
        learning_rule = AdamLearningRule(learning_rate=learning_rate)
        
        # Train the model and capture statistics
        stats, keys, run_time, fig_1, ax_1, fig_2, ax_2 = train_model_and_plot_stats(
            model, error, learning_rule, train_data, valid_data, num_epochs, stats_interval, notebook=False)
        
        # Store the results for each number of hidden units
        results[hidden_units] = {
            "stats": stats,
            "keys": keys,
            "fig_1": fig_1,  # Error plot (training vs validation)
            "fig_2": fig_2,  # Accuracy plot (training vs validation)
        }
    
    return results

# Set hyperparameters
input_dim, output_dim = 784, 47  # Input: 784 (28x28 image), Output: 47 (EMNIST classes)
learning_rate = 0.001  # Adam learning rate
num_epochs = 100  # Number of training epochs
stats_interval = 1  # Interval to record stats
batch_size = 100  # Batch size for training and validation

# Define the list of hidden unit sizes to evaluate (32, 64, 128)
hidden_units_list = [32, 64, 128]

# Train and evaluate models with different hidden unit sizes
results = evaluate_varying_hidden_units(
    hidden_units_list, input_dim, output_dim, learning_rate, num_epochs, stats_interval, train_data, valid_data
)


Training model with 32 hidden units.


Ep Prog: 100%|██████████| 1000/1000 [00:00<00:00, 1713.72it/s]
Epoch 1: 0.6s to complete
    error(train)=1.25e+00, acc(train)=6.53e-01, error(valid)=1.25e+00, acc(valid)=6.48e-01
Ep Prog: 100%|██████████| 1000/1000 [00:00<00:00, 1717.44it/s]
Epoch 2: 0.6s to complete
    error(train)=1.10e+00, acc(train)=6.92e-01, error(valid)=1.11e+00, acc(valid)=6.87e-01
Ep Prog: 100%|██████████| 1000/1000 [00:00<00:00, 1536.40it/s]
Epoch 3: 0.7s to complete
    error(train)=1.01e+00, acc(train)=7.16e-01, error(valid)=1.02e+00, acc(valid)=7.11e-01
Ep Prog: 100%|██████████| 1000/1000 [00:00<00:00, 1521.19it/s]
Epoch 4: 0.7s to complete
    error(train)=9.52e-01, acc(train)=7.27e-01, error(valid)=9.73e-01, acc(valid)=7.21e-01
Exp Prog:   4%|▍         | 4/100 [00:04<01:46,  1.10s/it]


KeyboardInterrupt: 

In [13]:
import matplotlib.pyplot as plt
import numpy as np

def plot_results(results, num_epochs, stats_interval):
    epochs = np.arange(1, num_epochs + 1, stats_interval)

    # Plot Accuracy (Training and Validation) for all hidden units configurations
    plt.figure(figsize=(10, 6))
    for hidden_units, res in results.items():
        stats = res['stats']
        keys = res['keys']
        
        # Plot training accuracy
        plt.plot(epochs, stats[1:, keys['acc(train)']], label=f'width {hidden_units}(train)')
        
        # Plot validation accuracy
        plt.plot(epochs, stats[1:, keys['acc(valid)']], label=f'width {hidden_units}(valid)', linestyle='--')
    
    plt.xlabel('Epoch number')
    plt.ylabel('Accuracy')
    # plt.title('Accuracy vs. Epochs for Varying Hidden Units')
    plt.legend(loc='lower right')
    plt.grid(True)
    plt.show()

    # Plot Error (Training and Validation) for all hidden units configurations
    plt.figure(figsize=(10, 6))
    for hidden_units, res in results.items():
        stats = res['stats']
        keys = res['keys']
        
        # Plot training error
        plt.plot(epochs, stats[1:, keys['error(train)']], label=f'width {hidden_units}(train)')
        
        # Plot validation error
        plt.plot(epochs, stats[1:, keys['error(valid)']], label=f'width {hidden_units}(valid)', linestyle='--')
    
    plt.xlabel('Epoch number')
    plt.ylabel('Error')
    # plt.title('Error vs. Epochs for Varying Hidden Units')
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

# Call the plotting function after training models
plot_results(results, num_epochs=num_epochs, stats_interval=stats_interval)

NameError: name 'results' is not defined

In [14]:
# Define a function to train models with varying number of hidden layers and track statistics
def evaluate_varying_hidden_layers(hidden_layer_counts, input_dim, output_dim, hidden_units, learning_rate, num_epochs, stats_interval, train_data, valid_data):
    results = {}
    
    for hidden_layers in hidden_layer_counts:
        print(f"\nTraining model with {hidden_layers} hidden layers, each with {hidden_units} units.")
        
        # Initialize weights and biases
        weights_init = GlorotUniformInit(rng=rng)
        biases_init = ConstantInit(0.)
        
        # Define model with the current number of hidden layers
        layers = []
        layers.append(AffineLayer(input_dim, hidden_units, weights_init, biases_init))  # First hidden layer
        layers.append(ReluLayer())
        
        # Add additional hidden layers based on `hidden_layers`
        for _ in range(1, hidden_layers):
            layers.append(AffineLayer(hidden_units, hidden_units, weights_init, biases_init))
            layers.append(ReluLayer())
        
        # Output layer
        layers.append(AffineLayer(hidden_units, output_dim, weights_init, biases_init))
        
        # Create model
        model = MultipleLayerModel(layers)
        
        # Set up the loss function (cross-entropy) and learning rule (Adam)
        error = CrossEntropySoftmaxError()
        learning_rule = AdamLearningRule(learning_rate=learning_rate)
        
        # Train the model and capture statistics
        stats, keys, run_time, fig_1, ax_1, fig_2, ax_2 = train_model_and_plot_stats(
            model, error, learning_rule, train_data, valid_data, num_epochs, stats_interval, notebook=False)
        
        # Store the results for each number of hidden layers
        results[hidden_layers] = {
            "stats": stats,
            "keys": keys,
            "fig_1": fig_1,  # Error plot (training vs validation)
            "fig_2": fig_2,  # Accuracy plot (training vs validation)
        }
    
    return results

# Set hyperparameters
input_dim, output_dim = 784, 47  # Input: 784 (28x28 image), Output: 47 (EMNIST classes)
hidden_units = 128  # Number of units in each hidden layer
learning_rate = 9e-4  # Adam learning rate
num_epochs = 100  # Number of training epochs
stats_interval = 1  # Interval to record stats
batch_size = 100  # Batch size for training and validation

# Define the number of hidden layers to evaluate (1, 2, and 3 hidden layers)
hidden_layer_counts = [1, 2, 3]

# Train and evaluate models with different numbers of hidden layers
results = evaluate_varying_hidden_layers(
    hidden_layer_counts, input_dim, output_dim, hidden_units, learning_rate, num_epochs, stats_interval, train_data, valid_data
)


Training model with 1 hidden layers, each with 128 units.


Exp Prog:   0%|          | 0/100 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
from mlp.penalties import L1Penalty,L2Penalty
def vary_penalty_train():
    penalties = [5e-4]
    learning_rate = 0.0001
    num_epochs = 100
    stats_interval = 1
    input_dim, output_dim, hidden_dim = 784, 47, 128
    models = []
    weights_init = GlorotUniformInit(rng=rng)
    biases_init = ConstantInit(0.)
    error = CrossEntropySoftmaxError()
    learning_rule = AdamLearningRule()

    print('For 1e-3 L1')
    for penalty in penalties:
        model = MultipleLayerModel([
        #DropoutLayer(rng,incl_prob=prob,share_across_batch=False),    
        AffineLayer(input_dim, hidden_dim, weights_init, biases_init, weights_penalty = L1Penalty(penalty), 
                    biases_penalty = L1Penalty(penalty)), 
        ReluLayer(),
        #DropoutLayer(rng,incl_prob=prob,share_across_batch=False),    
        AffineLayer(hidden_dim, hidden_dim, weights_init, biases_init, weights_penalty=L1Penalty(penalty), 
                   biases_penalty = L1Penalty(penalty)), 
        ReluLayer(),
        #DropoutLayer(rng,incl_prob=prob,share_across_batch=False),
        AffineLayer(hidden_dim, hidden_dim, weights_init, biases_init, weights_penalty=L1Penalty(penalty),
                   biases_penalty = L1Penalty(penalty)), 
        ReluLayer(),
        #DropoutLayer(rng,incl_prob=prob,share_across_batch=False),    
        AffineLayer(hidden_dim, output_dim, weights_init, biases_init, weights_penalty=L1Penalty(penalty),
                   biases_penalty = L1Penalty(penalty))])
        
        stats, keys, run_time, fig_1, ax_1, fig_2, ax_2 = train_model_and_plot_stats(model, error, learning_rule, train_data, 
                                 valid_data, num_epochs, stats_interval, notebook=True)
        
        models.append((stats,keys,'L1',penalty))
        
    print('For 1e-3 L2')
    for penalty in penalties:
        model = MultipleLayerModel([
        #DropoutLayer(rng,incl_prob=prob,share_across_batch=False),    
        AffineLayer(input_dim, hidden_dim, weights_init, biases_init, weights_penalty = L2Penalty(penalty),
                   biases_penalty = L2Penalty(penalty)), 
        ReluLayer(),
        #DropoutLayer(rng,incl_prob=prob,share_across_batch=False),    
        AffineLayer(hidden_dim, hidden_dim, weights_init, biases_init, weights_penalty=L2Penalty(penalty),
                   biases_penalty = L2Penalty(penalty)), 
        ReluLayer(),
        #DropoutLayer(rng,incl_prob=prob,share_across_batch=False),
        AffineLayer(hidden_dim, hidden_dim, weights_init, biases_init, weights_penalty=L2Penalty(penalty),
                   biases_penalty = L2Penalty(penalty)), 
        ReluLayer(),
        #DropoutLayer(rng,incl_prob=prob,share_across_batch=False),    
        AffineLayer(hidden_dim, output_dim, weights_init, biases_init, weights_penalty=L2Penalty(penalty),
                   biases_penalty = L2Penalty(penalty))])
        
        stats_2, keys_2, run_time_2, fig2_1, ax2_1, fig2_2, ax2_2 = train_model_and_plot_stats(
            model, error, learning_rule, train_data, valid_data, num_epochs, stats_interval, notebook=False)
        
        
        models.append((stats_2,keys_2,'L2',penalty))
    
        
    #fig_1,_,_ = plot_with_generalization_gap(models)
    #fig_1.savefig('penalty.svg',format='svg')
    return models

models_penalties = vary_penalty_train()

In [ ]:
from mlp.layers import DropoutLayer
def vary_dropout_train():
    incl_probs = [0.7]
    learning_rate = 0.1
    num_epochs = 100
    stats_interval = 1
    input_dim, output_dim,hidden_dim = 784, 47, 128
    models = []
    weights_init = GlorotUniformInit(rng=rng)
    biases_init = ConstantInit(0.)
    error = CrossEntropySoftmaxError()
    learning_rule = AdamLearningRule()
    for prob in incl_probs:
        model = MultipleLayerModel([
        DropoutLayer(rng,incl_prob=prob,share_across_batch=False),    
        AffineLayer(input_dim, hidden_dim, weights_init, biases_init), 
        ReluLayer(),

        DropoutLayer(rng,incl_prob=prob,share_across_batch=False),    
        AffineLayer(hidden_dim, hidden_dim, weights_init, biases_init), 
        ReluLayer(),

        DropoutLayer(rng,incl_prob=prob,share_across_batch=False),
        AffineLayer(hidden_dim, hidden_dim, weights_init, biases_init), 
        ReluLayer(),
        
        DropoutLayer(rng,incl_prob=prob,share_across_batch=False),    
        AffineLayer(hidden_dim, output_dim, weights_init, biases_init)])
        
        stats, keys, run_time, fig_1, ax_1, fig_2, ax_2 = train_model_and_plot_stats(model, error, learning_rule, train_data, 
                                 valid_data, num_epochs, stats_interval, notebook=True)
        
        models.append((stats,keys))
        
    #fig_1,_,fig_2,_ = plot(models,stats_interval,value_to_be_varried="dropout")
    #fig_1.savefig('error-dropout.svg',format='svg')
    #fig_2.savefig('accuracy-dropout.svg',format='svg')
    return models
    
model_dropouts = vary_dropout_train() 

In [ ]:
from mlp.data_providers import EMNISTDataProvider
def vary_dropout_train():
    incl_probs = [0.1]
    num_epochs = 100
    stats_interval = 1
    input_dim, output_dim,hidden_dim = 784, 47, 128
    models = []
    weights_init = GlorotUniformInit(rng=rng)
    biases_init = ConstantInit(0.)
    error = CrossEntropySoftmaxError()
    learning_rule = AdamLearningRule()

    # Use smoothed labels in data provider
    train_data = EMNISTDataProvider(which_set='train', batch_size=100, smooth_labels=True)
    valid_data = EMNISTDataProvider(which_set='valid', batch_size=100, smooth_labels=True)

    for prob in incl_probs:
        model = MultipleLayerModel([
        AffineLayer(input_dim, hidden_dim, weights_init, biases_init), 
        ReluLayer(),
   
        AffineLayer(hidden_dim, hidden_dim, weights_init, biases_init), 
        ReluLayer(),

        AffineLayer(hidden_dim, hidden_dim, weights_init, biases_init), 
        ReluLayer(),
        
        AffineLayer(hidden_dim, output_dim, weights_init, biases_init)])
        
        stats, keys, run_time, fig_1, ax_1, fig_2, ax_2 = train_model_and_plot_stats(model, error, learning_rule, train_data, 
                                 valid_data, num_epochs, stats_interval, notebook=True)
        
        models.append((stats,keys))
        
    #fig_1,_,fig_2,_ = plot(models,stats_interval,value_to_be_varried="dropout")
    #fig_1.savefig('error-dropout.svg',format='svg')
    #fig_2.savefig('accuracy-dropout.svg',format='svg')
    return models
    
model_dropouts = vary_dropout_train() 

In [28]:
from mlp.penalties import L1Penalty,L2Penalty
def penalty_train_new():
    penalties = [5e-4,1e-3,5e-3,5e-2]
    num_epochs = 100
    stats_interval = 1
    input_dim, output_dim,hidden_dim = 784, 47, 128
    models = []
    weights_init = GlorotUniformInit(rng=rng)
    biases_init = ConstantInit(0.)
    error = CrossEntropySoftmaxError()
    learning_rule = AdamLearningRule(learning_rate=0.0001)
    for penalty in penalties:
        model = MultipleLayerModel([
        #DropoutLayer(rng,incl_prob=prob,share_across_batch=False),    
        AffineLayer(input_dim, hidden_dim, weights_init, biases_init, weights_penalty = L1Penalty(penalty)), 
        ReluLayer(),
        #DropoutLayer(rng,incl_prob=prob,share_across_batch=False),    
        AffineLayer(hidden_dim, hidden_dim, weights_init, biases_init, weights_penalty=L1Penalty(penalty)), 
        ReluLayer(),
        #DropoutLayer(rng,incl_prob=prob,share_across_batch=False),
        AffineLayer(hidden_dim, hidden_dim, weights_init, biases_init, weights_penalty=L1Penalty(penalty)), 
        ReluLayer(),
        #DropoutLayer(rng,incl_prob=prob,share_across_batch=False),    
        AffineLayer(hidden_dim, output_dim, weights_init, biases_init, weights_penalty=L1Penalty(penalty))])
        
        stats, keys, run_time, fig_1, ax_1, fig_2, ax_2 = train_model_and_plot_stats(model, error, learning_rule, train_data, 
                                 valid_data, num_epochs, stats_interval, notebook=True)
        
        models.append((stats,keys,'L1',penalty))
        
    for penalty in penalties:
        model = MultipleLayerModel([
        #DropoutLayer(rng,incl_prob=prob,share_across_batch=False),    
        AffineLayer(input_dim, hidden_dim, weights_init, biases_init, weights_penalty = L2Penalty(penalty)), 
        ReluLayer(),
        #DropoutLayer(rng,incl_prob=prob,share_across_batch=False),    
        AffineLayer(hidden_dim, hidden_dim, weights_init, biases_init, weights_penalty=L2Penalty(penalty)), 
        ReluLayer(),
        #DropoutLayer(rng,incl_prob=prob,share_across_batch=False),
        AffineLayer(hidden_dim, hidden_dim, weights_init, biases_init, weights_penalty=L2Penalty(penalty)), 
        ReluLayer(),
        #DropoutLayer(rng,incl_prob=prob,share_across_batch=False),    
        AffineLayer(hidden_dim, output_dim, weights_init, biases_init, weights_penalty=L2Penalty(penalty))])
        
        stats_2, keys_2, run_time_2, fig2_1, ax2_1, fig2_2, ax2_2 = train_model_and_plot_stats(model, error, learning_rule, train_data, 
                                 valid_data, num_epochs, stats_interval, notebook=True)
        
        models.append((stats_2,keys_2,'L2',penalty))
    
        
    #fig_1,_,_ = plot_with_generalization_gap(models)
    #fig_1.savefig('penalty.svg',format='svg')
    return models

models_penalties_new = penalty_train_new()

Epoch 27: 13.2s to complete
    error(train)=4.64e-01, acc(train)=8.50e-01, error(valid)=5.26e-01, acc(valid)=8.30e-01


  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 28: 12.7s to complete
    error(train)=4.55e-01, acc(train)=8.52e-01, error(valid)=5.19e-01, acc(valid)=8.33e-01


  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 29: 13.1s to complete
    error(train)=4.56e-01, acc(train)=8.52e-01, error(valid)=5.20e-01, acc(valid)=8.31e-01


...

Epoch 98: 3.4s to complete
    error(train)=2.17e+00, acc(train)=4.14e-01, error(valid)=2.17e+00, acc(valid)=4.10e-01


  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 99: 3.2s to complete
    error(train)=2.16e+00, acc(train)=4.19e-01, error(valid)=2.16e+00, acc(valid)=4.15e-01


  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 100: 3.4s to complete
    error(train)=2.17e+00, acc(train)=4.15e-01, error(valid)=2.17e+00, acc(valid)=4.10e-01


In [ ]:
from mlp.layers import DropoutLayer
def vary_dropout_new():
    incl_probs = [0.97,0.85,0.7,0.6]
    num_epochs = 100
    stats_interval = 1
    input_dim, output_dim,hidden_dim = 784, 47, 128
    models = []
    weights_init = GlorotUniformInit(rng=rng)
    biases_init = ConstantInit(0.)
    error = CrossEntropySoftmaxError()
    learning_rule = AdamLearningRule(learning_rate=0.0001)
    for prob in incl_probs:
        model = MultipleLayerModel([
        DropoutLayer(rng,incl_prob=prob,share_across_batch=False),    
        AffineLayer(input_dim, hidden_dim, weights_init, biases_init), 
        ReluLayer(),

        DropoutLayer(rng,incl_prob=prob,share_across_batch=False),    
        AffineLayer(hidden_dim, hidden_dim, weights_init, biases_init), 
        ReluLayer(),

        DropoutLayer(rng,incl_prob=prob,share_across_batch=False),
        AffineLayer(hidden_dim, hidden_dim, weights_init, biases_init), 
        ReluLayer(),
        
        DropoutLayer(rng,incl_prob=prob,share_across_batch=False),    
        AffineLayer(hidden_dim, output_dim, weights_init, biases_init)])
        
        stats, keys, run_time, fig_1, ax_1, fig_2, ax_2 = train_model_and_plot_stats(model, error, learning_rule, train_data, 
                                 valid_data, num_epochs, stats_interval, notebook=True)
        
        models.append((stats,keys))
        
    
    return models
    
model_dropouts_new = vary_dropout_new() 

In [6]:
def plot_with_generalization_gap(models,notebook=True):
    #Value to be varried could be width,depth,L1/L2
    fig = plt.figure(figsize=(8, 4))
    ax_1 = fig.add_subplot(111, xscale='log')
    penalties = [5e-4,1e-3,5e-3,5e-2]
    valid_acc_l1 = []
    valid_acc_l2 = []
    gen_gap_l1 = []
    gen_gap_l2 = []
    for stats,keys,penalty_name,penalty_val in models: #penalty is either L1 or L2
        if penalty_name == "L1":
            valid_acc_l1.append(stats[1:,keys["acc(valid)"]][-1])
        elif penalty_name == "L2":
            valid_acc_l2.append(stats[1:,keys["acc(valid)"]][-1])
        
    lns1 = ax_1.plot(penalties,valid_acc_l1,label='L1 Val Acc')
    lns2 = ax_1.plot(penalties,valid_acc_l2,label='L2 Val Acc')
            
    ax_2 = ax_1.twinx()
    
    
    for stats,keys,penalty_name,penalty_val in models: #penalty is either L1 or L2
        if penalty_name=="L1":
            gen_gap_l1.append(abs(stats[1:,keys['error(valid)']][-1] - stats[1:,keys['error(train)']][-1]))
        elif penalty_name=="L2":
            gen_gap_l2.append(abs(stats[1:,keys['error(valid)']][-1] - stats[1:,keys['error(train)']][-1]))
        
    lns3 = ax_2.plot(penalties,gen_gap_l1,'--',label='L1 Gap')
    lns4 = ax_2.plot(penalties,gen_gap_l2,'--',label='L2 Gap')
    lns = lns1 + lns2 + lns3 + lns4
    labels = [l.get_label() for l in lns]
    ax_1.legend(lns,labels,loc=0)
    ax_1.set_xlabel('Weight decay value')
    ax_1.set_ylabel('Accuracy')
    ax_2.set_xlabel('Weight decay value')
    ax_2.set_ylabel("Generalization gap")
    ax_2.set_ylim(0.0,1.0)
    ax_1.set_ylim(0.0,1.0)
    #ax_1.set_xlim(0,1e-1)
    ax_1.set_xticks(penalties)
    #ax_1.set_xticklabels([0,'$10^{-5}$','$10^{-4}$','$10^{-3}$','$10^{-2}$','$10^{-1}$'])
    #ax_1.xaxis.set_minor_locator(plt.MultipleLocator(1e-5))
    
    return fig,ax_1,ax_2    

In [18]:
def plot_dropout(models,notebook=True):
    #Value to be varried could be width,depth,L1/L2
    fig = plt.figure(figsize=(8, 4))
    ax_1 = fig.add_subplot(111)
    
    dropouts_val = [0.97,0.85,0.7,0.6]
    dropout_val_accs = []
    dropout_val_gaps = []
    
    
    #for k in ['error(train)','error(valid)']:
    for stats,keys in models: #penalty is either L1 or L2
        dropout_val_accs.append(stats[1:,keys['acc(valid)']][-1])
        #generalization_gap = abs(stats[1:,keys['error(valid)']] - stats[1:,keys['error(train)']]) 
        
    lns1 = ax_1.plot(dropouts_val,dropout_val_accs,label='Val Acc')
            
    ax_2 = ax_1.twinx()
    
    #for k in ['acc(train)','acc(valid)']:
    for stats,keys in models: #penalty is either L1 or L2
        gap = abs(stats[1:,keys['error(valid)']][-1] - stats[1:,keys['error(train)']][-1])
        dropout_val_gaps.append(gap)
        
    
    lns2 = ax_2.plot(dropouts_val,dropout_val_gaps,'-b',label='Gap')
    lns = lns1 + lns2
    labels = [l.get_label() for l in lns]
    
            
    ax_1.legend(lns,labels,loc=0)
    ax_1.set_xlabel('Dropout value')
    ax_1.set_ylabel('Accuracy')
    ax_2.set_xlabel('Dropout value')
    ax_2.set_ylabel("Generalization gap")
    ax_2.set_ylim(0.0,1.0)
    ax_1.set_ylim(0.0,1.0)
    #ax_1.set_xlim(1e-5,1e-1)
    ax_1.set_xticks(dropouts_val)
    ax_1.set_xticklabels(dropouts_val,fontsize=11)
    
    return fig,ax_1,ax_2  

In [21]:
fig,_,_ = plot_dropout(model_dropouts_new)
fig.savefig('dropout.png',format='png')

In [29]:
fig,_,_ = plot_with_generalization_gap(models_penalties_new)
fig.savefig('penalties-no-bias.png',format='png')